In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras.models import Model

In [3]:
BATCH_SIZE = 32
NUM_EPOCHS = 10
STEPS_PER_EPOCH = 30
START_LEARNING_RATE = 0.01

In [4]:
def dataset_input_fn(filenames, batch_size):
    dataset = tf.data.TFRecordDataset(filenames)

    # Use `tf.parse_single_example()` to extract data from a `tf.Example`
    # protocol buffer, and perform any additional per-record preprocessing.
    def parser(record):
        keys_to_features = {
            "board": tf.FixedLenFeature((169, ), tf.int64,
                                        default_value=tf.zeros([], dtype=tf.int64)),
            "winner": tf.FixedLenFeature((), tf.int64, default_value=0),
        }
        
        parsed = tf.parse_single_example(record, keys_to_features)
        
        winner = parsed['winner']
        winner = tf.cast(winner, tf.int8)
        
        board = parsed['board']
        board = tf.reshape(board, (13, 13))
        board = tf.cast(board, tf.int8)

        return {"board": board}, winner

    # Use `Dataset.map()` to build a pair of a feature dictionary and a label
    # tensor for each example.
    dataset = dataset.map(parser)
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size)

    # Each element of `dataset` is tuple containing a dictionary of features
    # (in which each value is a batch of values for that feature), and a batch of
    # labels.
    return dataset

In [23]:
def model_function():
    # a layer instance is callable on a tensor, and returns a tensor
    inputs = Input(shape=(13, 13, 1))
    x = Conv2D(32,
               kernel_size=(5, 5),
               strides=(1, 1),
               activation='relu')(inputs)
    x = MaxPool2D(pool_size=(2, 2),
                  strides=(2, 2))(x)
    x = Conv2D(64,
               kernel_size=(3, 3),
               activation='relu')(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(1, activation='softmax')(x)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [24]:
dataset = dataset_input_fn(['data/full_dataset.tfrecord'], BATCH_SIZE)
model = model_function()

In [26]:
# Create a variable to hold the global_step.
    
    sess.run(global_step_tensor.initializer)
    tf.keras.backend.set_session(sess)

    model.fit(dataset, epochs=NUM_EPOCHS, steps_per_epoch=STEPS_PER_EPOCH)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10 into data/checkpoints/model.ckpt.


RuntimeError: Graph is finalized and cannot be modified.